In [ ]:
import torch
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/detection/baseline/mmdetection/configs/object/trash.py')

# data root 설정
root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = '/opt/ml/stratified_kfold/cv_train_1.json' # train json 정보

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = '/opt/ml/stratified_kfold/cv_val_1.json' # train json 정보

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

cfg.data.samples_per_gpu = 1 # batch_size

# seed 설정
cfg.seed = 2021

# gpu_ids 설정
cfg.gpu_ids = [0]

# work_dir 경로 설정
cfg.work_dir = './work_dirs/swinL_final'

# model num_classes 설정
cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)




In [ ]:
#print(cfg.data.train)
#torch.cuda.empty_cache()

In [ ]:
cfg.pretrained = '/opt/ml/swin_large_patch4_window7_224_22k.pth'

In [ ]:
datasets = [build_dataset(cfg.data.train)] # dataset 정의
datasets[0]

In [ ]:
model = build_detector(cfg.model) # 모델 build 및 pretrained network 불러오기
model.init_weights()

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True) # 모델 학습